### Intro
Train the small model - CIFAR-10 Analysis
Which normalization to use, imagenet or cifar10? when fine tuning, I could not find a solid report on this.

In [ ]:
#!/usr/bin/env python3
import os
import sys
import json
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from torchvision import datasets, transforms, models
from torchvision.models import MobileNet_V3_Small_Weights
from torch.utils.data import DataLoader
from tqdm import tqdm

# --- Reproducibility Setup ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
# When running on the CuDNN backend, two further options must be set
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Paths
DATA_DIR = '/root/arcade/data/cifar10_split'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VAL_DIR   = os.path.join(DATA_DIR, 'validation')
TEST_DIR  = os.path.join(DATA_DIR, 'test')

MODEL_DIR   = '/root/arcade/final_scripts/final_models'
RESULTS_DIR = '/root/arcade/final_scripts/final_results'
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# GPU setup
try:
    if not torch.cuda.is_available():
        raise RuntimeError("GPU not available")
    device = torch.device("cuda")
    print(f"Using device: {device}")
except Exception as e:
    print(f"Error initializing GPU: {e}", file=sys.stderr)
    sys.exit(1)

# --- Main experiment loop ---
all_results = []
normalization_configs = {
    # these are from => https://stackoverflow.com/questions/66678052/how-to-calculate-the-mean-and-the-std-of-cifar10-data
    'cifar10': {
        'mean': (0.4914, 0.4822, 0.4465),
        'std': (0.247, 0.243, 0.261)
    },
    'imagenet': {
        'mean': (0.485, 0.456, 0.406),
        'std': (0.229, 0.224, 0.225)
    }
}

RESIZE_DIM = (224, 224)

for norm_type, params in normalization_configs.items():
    print(f"\n{'='*50}")
    print(f"RUNNING EXPERIMENT WITH NORMALIZATION: {norm_type.upper()}")
    print(f"{'='*50}\n")

    # 1. Set up transforms for the current experiment
    transform = transforms.Compose([
        transforms.Resize(RESIZE_DIM),
        transforms.ToTensor(),
        transforms.Normalize(mean=params['mean'], std=params['std']),
    ])

    # 2. Create datasets and dataloaders
    train_ds = datasets.ImageFolder(TRAIN_DIR, transform=transform)
    val_ds   = datasets.ImageFolder(VAL_DIR,   transform=transform)
    test_ds  = datasets.ImageFolder(TEST_DIR,  transform=transform)

    BATCH_SIZE = 256
    NUM_WORKERS = 4
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                              num_workers=NUM_WORKERS, pin_memory=True)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                              num_workers=NUM_WORKERS, pin_memory=True)

    # 3. Re-initialize the model and optimizer to start fresh
    weights = MobileNet_V3_Small_Weights.DEFAULT
    model = models.mobilenet_v3_small(weights=weights)
    in_feats = model.classifier[3].in_features
    model.classifier[3] = nn.Linear(in_feats, len(train_ds.classes))
    model = model.to(device)

    LR = 1e-3
    NUM_EPOCHS = 20
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)

    best_val_acc = 0.0
    history = []

    # 4. Training loop
    for epoch in range(1, NUM_EPOCHS + 1):
        model.train()
        train_loss, correct, total = 0.0, 0, 0
        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} [Train]"):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(targets).sum().item()
            total += targets.size(0)
        train_loss /= total
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} [Val]"):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                _, preds = outputs.max(1)
                correct += preds.eq(targets).sum().item()
                total += targets.size(0)
        val_loss /= total
        val_acc = correct / total

        history.append({'epoch': epoch, 'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': val_loss, 'val_acc': val_acc})
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), os.path.join(MODEL_DIR, f'best_model_{norm_type}.pth'))
        print(f"Epoch {epoch}: Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    # 5. Load best model and test
    model.load_state_dict(torch.load(os.path.join(MODEL_DIR, f'best_model_{norm_type}.pth')))
    model.eval()
    test_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc="Testing"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(targets).sum().item()
            total += targets.size(0)
    test_loss /= total
    test_acc = correct / total
    
    print(f"\nDone with {norm_type}! Best Val Acc: {best_val_acc:.4f}, Test Acc: {test_acc:.4f}")

    # 6. Store results for this run
    results = {
        'normalization_type': norm_type,
        'resize_size': list(RESIZE_DIM), # Added this line
        'normalization_mean': params['mean'],
        'normalization_std': params['std'],
        'best_val_acc': best_val_acc,
        'test_loss': test_loss,
        'test_acc': test_acc,
        'history': history,
        'model_details': {
            'model': 'mobilenet_v3_small',
            'weights': 'MobileNet_V3_Small_Weights.DEFAULT',
            'batch_size': BATCH_SIZE,
            'learning_rate': LR,
            'num_epochs': NUM_EPOCHS,
            'optimizer': 'Adam',
            'seed': SEED
        }
    }
    all_results.append(results)

# 7. Save all combined results to a single JSON file
output_file = os.path.join(RESULTS_DIR, 'normalization_compar.json')
with open(output_file, 'w') as f:
    json.dump(all_results, f, indent=4)

print(f"\nAll experiments complete. Combined results saved to {output_file}")

Using device: cuda

RUNNING EXPERIMENT WITH NORMALIZATION: CIFAR10



Epoch 1/20 [Val]: 100%|██████████| 47/47 [00:04<00:00,  9.80it/s]


Epoch 1: Train Acc: 0.8696, Val Acc: 0.8663


Epoch 2/20 [Val]: 100%|██████████| 47/47 [00:04<00:00, 10.00it/s]


Epoch 2: Train Acc: 0.9486, Val Acc: 0.8921


...

Epoch 19: Train Acc: 0.9946, Val Acc: 0.9225


Epoch 20/20 [Val]: 100%|██████████| 47/47 [00:04<00:00,  9.57it/s]


Epoch 20: Train Acc: 0.9923, Val Acc: 0.8869


Testing: 100%|██████████| 47/47 [00:04<00:00,  9.52it/s]


Done with imagenet! Best Val Acc: 0.9251, Test Acc: 0.9267

All experiments complete. Combined results saved to /root/arcade/final_scripts/final_results/comparison_results.json


- Imagenet Normalization is better, we should go with that one.
- Model path is: /root/arcade/final_scripts/final_models/best_model_imagenet.pth

We evalaute the baseline performance on cifar-10-c.

In [ ]:
#!/usr/bin/env python3
import os
import sys
import torch
import numpy as np
import json # <-- Added this import
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# -- Step 1: Set the correct path and normalization --
MODEL_PATH = '/root/arcade/final_scripts/final_models/best_model_imagenet.pth'
NORMALIZATION = {
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225]
}

# --- Other Parameters ---
CIFAR_C_DIR = '/root/arcade/data/CIFAR-10-C'
RESULTS_DIR = '/root/arcade/final_scripts/final_results' # <-- Added results path
BATCH_SIZE = 256
NUM_WORKERS = 4
RESIZE_DIM = (224, 224)

# GPU setup
try:
    if not torch.cuda.is_available():
        raise RuntimeError("GPU not available")
    device = torch.device("cuda")
    print(f"Using device: {device}")
except Exception as e:
    print(f"Error initializing GPU: {e}", file=sys.stderr)
    sys.exit(1)

# --- Define a custom Dataset for .npy files ---
class CustomNumpyDataset(Dataset):
    """Custom Dataset for loading data from numpy arrays"""
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = torch.from_numpy(labels).long()
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = transforms.ToPILImage()(image)
        if self.transform:
            image = self.transform(image)
        return image, label

# --- Prepare Model ---
model = models.mobilenet_v3_small(weights=None)
in_feats = model.classifier[3].in_features
model.classifier[3] = torch.nn.Linear(in_feats, 10)

if not os.path.exists(MODEL_PATH):
    print(f"Error: Model path not found at {MODEL_PATH}", file=sys.stderr)
    sys.exit(1)
    
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(device)
model.eval()
print(f"Model loaded from {MODEL_PATH}")

# --- Prepare Data Transforms ---
eval_transform = transforms.Compose([
    transforms.Resize(RESIZE_DIM),
    transforms.ToTensor(),
    transforms.Normalize(mean=NORMALIZATION['mean'], std=NORMALIZATION['std']),
])

# --- Evaluation Loop ---
labels_path = os.path.join(CIFAR_C_DIR, 'labels.npy')
if not os.path.exists(labels_path):
    print(f"Error: labels.npy not found in {CIFAR_C_DIR}", file=sys.stderr)
    sys.exit(1)
labels = np.load(labels_path)

corruption_files = [f for f in os.listdir(CIFAR_C_DIR) if f.endswith('.npy') and f != 'labels.npy']
corruption_results = {}

print("\nStarting evaluation on CIFAR-10-C corruptions...")

for corruption_file in sorted(corruption_files):
    images_path = os.path.join(CIFAR_C_DIR, corruption_file)
    images = np.load(images_path)
    
    dataset = CustomNumpyDataset(images, labels, transform=eval_transform)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in tqdm(loader, desc=f"Testing {corruption_file.replace('.npy', '')}"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
    accuracy = 100 * correct / total
    corruption_results[corruption_file.replace('.npy', '')] = accuracy

# --- Print Final Results ---
print("\n--- CIFAR-10-C Evaluation Results ---")
for corruption_type, acc in corruption_results.items():
    print(f"{corruption_type:<20} | Accuracy: {acc:.2f}%")
print("---------------------------------------")

# --- Save results to JSON file --- # <-- ADDED THIS ENTIRE BLOCK
final_output = {
    'model_evaluated': MODEL_PATH,
    'evaluation_dataset': 'CIFAR-10-C',
    'corruption_accuracies': corruption_results
}
os.makedirs(RESULTS_DIR, exist_ok=True)
output_filepath = os.path.join(RESULTS_DIR, 'cifar_c_results.json')
with open(output_filepath, 'w') as f:
    json.dump(final_output, f, indent=4)

print(f"\nEvaluation results saved to {output_filepath}")